# Αυτό το notebook αφαιρεί πρατήρια που έχουν μείνει ως σήμεια στον χάρτη (έχουν κλείσει) και υπάρχουν καινούργια δίπλα τους 

##### 1. Βιβλιοθήκες

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import DBSCAN
from scipy.spatial.distance import cdist
from pathlib import Path
import folium
from folium.plugins import MarkerCluster
import warnings
warnings.filterwarnings('ignore')

# Configuration
INPUT_FILE = "/Users/geo/Desktop/fuelstation-detection-thesis/data/ALL χιλιομετικές διευθύνσεις.xlsx"
OUTPUT_FILE = "/Users/geo/Desktop/fuelstation-detection-thesis/data/ALL_cleaned.xlsx"
DISTANCE_THRESHOLD = 20   # metra
ID_COLUMN = 'gasStationID'
SHEET_NAME = 0

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print(f"Distance threshold: {DISTANCE_THRESHOLD}m")

Distance threshold: 20m


#### 2. Data Loading

In [3]:
df_original = pd.read_excel(INPUT_FILE, sheet_name=SHEET_NAME)

print(f"\nDataset info:")
print(f"Total records: {len(df_original)}")
print(f"\nColumns: {list(df_original.columns)}")
df_original.head()

df = df_original.copy()

# Kraταme mono ta ROOFTOP pratiria
initial_len = len(df)
df = df[df['locationType'] == 'ROOFTOP'].copy()
print(f"Mono osa einai ROOFTOP: {len(df)} records (removed {initial_len - len(df)} non-ROOFTOP)")
print(f"Working with {len(df)} records")


Dataset info:
Total records: 1278

Columns: ['gasStationID', 'gasStationAddress', 'gasStationLat', 'gasStationLong', 'locationType', 'ddName', 'municipalityName', 'countyName']
Mono osa einai ROOFTOP: 1262 records (removed 16 non-ROOFTOP)
Working with 1262 records


#### 3. Υπολογισμός αποστάσεων και cluster 

In [6]:
def remove_nearby_duplicates(df, distance_threshold=10, id_column='gasStationID'):

    print("\n \n \n")
    print(f"REMOVING DUPLICATE FUEL STATIONS")
    print("\n \n \n")
    print(f"Initial stations: {len(df)}")
    print(f"Distance threshold: {distance_threshold}m")

    df_work = df.copy()

    # Metatropi lat/lon se metra (Euclidean approximation)
    mean_lat = df_work['gasStationLat'].mean()
    lat_m = df_work['gasStationLat'].values * 111320  # 1 degree lat = 111.32 km
    lon_m = df_work['gasStationLong'].values * 111320 * np.cos(np.radians(mean_lat))
    
    coords = np.column_stack([lat_m, lon_m])

    # Ypologismos apostaseon (Euclidean)
    print("\nCalculating distances...")
    distance_matrix = cdist(coords, coords, metric='euclidean')

    # DBSCAN clustering
    print("Finding clusters...")
    clustering = DBSCAN(eps=distance_threshold, min_samples=1, metric='precomputed')
    df_work['cluster'] = clustering.fit_predict(distance_matrix)

    # Statistika
    n_clusters = df_work['cluster'].nunique()
    clusters_with_duplicates = df_work['cluster'].value_counts()
    clusters_with_duplicates = clusters_with_duplicates[clusters_with_duplicates > 1]

    print(f"\nTotal clusters: {n_clusters}")
    print(f"Clusters with duplicates: {len(clusters_with_duplicates)}")

    # Epilogi pratiriwn
    keep_indices = []
    remove_indices = []

    for cluster_id in df_work['cluster'].unique():
        cluster_members = df_work[df_work['cluster'] == cluster_id]

        if len(cluster_members) == 1:
            keep_indices.append(cluster_members.index[0])
        else:
            max_id_idx = cluster_members[id_column].idxmax()
            keep_indices.append(max_id_idx)

            for idx in cluster_members.index:
                if idx != max_id_idx:
                    remove_indices.append(idx)

    # Dimiourgia cleaned kai removed DataFrames
    cleaned_df = df.loc[keep_indices].copy().reset_index(drop=True)
    removed_df = df.loc[remove_indices].copy().reset_index(drop=True)

    # Prosthiki replaced_by_id sto removed_df
    if len(removed_df) > 0:
        removed_df['replaced_by_id'] = None
        for idx in remove_indices:
            cluster_id = df_work.loc[idx, 'cluster']
            cluster_members = df_work[df_work['cluster'] == cluster_id]
            kept_id = cluster_members[id_column].max()
            removed_df.loc[removed_df.index[remove_indices.index(idx)], 'replaced_by_id'] = kept_id

    # Prosthiki cluster info sta cleaned data
    cleaned_df['cluster'] = df_work.loc[keep_indices, 'cluster'].values

    # Statistics
    stats = {
        'original_count': len(df),
        'kept_count': len(cleaned_df),
        'removed_count': len(removed_df),
        'clusters_total': n_clusters,
        'clusters_with_duplicates': len(clusters_with_duplicates),
        'distance_threshold': distance_threshold,
        'retention_rate': len(cleaned_df) / len(df) * 100
    }

    print("\n")
    print(f"RESULTS")
    print("\n")
    print(f"Initial stations:  {stats['original_count']}")
    print(f"Kept stations:     {stats['kept_count']}")
    print(f"Removed stations:  {stats['removed_count']}")
    print("\n")

    return cleaned_df, removed_df, stats, df_work

# Ektelesi
cleaned_df, removed_df, stats, df_work = remove_nearby_duplicates(
    df,
    distance_threshold=DISTANCE_THRESHOLD,
    id_column=ID_COLUMN 
)


 
 

REMOVING DUPLICATE FUEL STATIONS

 
 

Initial stations: 1262
Distance threshold: 20m

Calculating distances...
Finding clusters...

Total clusters: 1026
Clusters with duplicates: 193


RESULTS


Initial stations:  1262
Kept stations:     1026
Removed stations:  236




In [7]:
# Elegxos gia exact duplicates STO CLEANED DATASET

print("CHECKING FOR REMAINING DUPLICATES IN CLEANED DATA")
print("\n")

duplicate_coords = cleaned_df[cleaned_df.duplicated(subset=['gasStationLat', 'gasStationLong'], keep=False)]

if len(duplicate_coords) > 0:
    print(f"WARNING: Found {len(duplicate_coords)} records with duplicate coordinates!")
    print("\nGrouped by coordinates:")
    for coords, group in duplicate_coords.groupby(['gasStationLat', 'gasStationLong']):
        print(f"\nCoordinates: Lat={coords[0]:.6f}, Lon={coords[1]:.6f}")
        print(f"  IDs: {group[ID_COLUMN].tolist()}")
        print(f"  Addresses: {group['gasStationAddress'].tolist()}")
else:
    print("SUCCESS: No duplicate coordinates found in cleaned dataset ")
print("\n")

CHECKING FOR REMAINING DUPLICATES IN CLEANED DATA


SUCCESS: No duplicate coordinates found in cleaned dataset 




### Αποτελέσματα

#### Χάρτης με τα καινούργια βενζινάδικα

In [18]:
# Ypologismos kentrou tou xarti
center_lat = df['gasStationLat'].mean()
center_lon = df['gasStationLong'].mean()

# Dimiourgia base map
m = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=10,
    tiles='OpenStreetMap'
)

# Dimiourgia FeatureGroup gia kept stations
kept_group = folium.FeatureGroup(name='Kept Stations (Green)')

# Prosthiki kept stations (green markers)
for idx, row in cleaned_df.iterrows():
    folium.CircleMarker(
        location=[row['gasStationLat'], row['gasStationLong']],
        radius=5,  # MISO APO 10
        popup=f"ID: {row[ID_COLUMN]}<br>Address: {row.get('gasStationAddress', 'N/A')}",
        color='green',
        fill=True,
        fillColor='green',
        fillOpacity=0.7,
        weight=2  # stroke-width
    ).add_to(kept_group)

kept_group.add_to(m)

# Dimiourgia FeatureGroup gia removed stations
if len(removed_df) > 0:
    removed_group = folium.FeatureGroup(name='Removed Stations (Red)')
    
    for idx, row in removed_df.iterrows():
        folium.CircleMarker(
            location=[row['gasStationLat'], row['gasStationLong']],
            radius=4,  # MISO APO 8
            popup=f"ID: {row[ID_COLUMN]}<br>Replaced by: {row['replaced_by_id']}<br>Address: {row.get('gasStationAddress', 'N/A')}",
            color='red',
            fill=True,
            fillColor='red',
            fillOpacity=0.6,
            weight=2  # stroke-width
        ).add_to(removed_group)
    
    removed_group.add_to(m)

# Prosthiki layer control
folium.LayerControl().add_to(m)

# Apothikeusi map
map_path = Path(OUTPUT_FILE).parent / 'fuel_stations_map_cleaned.html'
m.save(str(map_path))
print(f"Map saved to: {map_path}")

m

Map saved to: /Users/geo/Desktop/fuelstation-detection-thesis/data/fuel_stations_map_cleaned.html


In [14]:
# Apothikeusi Excel
print(f"\n Saving results to: {OUTPUT_FILE}")

with pd.ExcelWriter(OUTPUT_FILE, engine='openpyxl') as writer:
    # Cleaned data
    cleaned_df.to_excel(writer, sheet_name='Cleaned', index=False)
    
    # Removed data
    if len(removed_df) > 0:
        removed_df.to_excel(writer, sheet_name='Removed', index=False)
    
    # Statistics
    stats_summary = pd.DataFrame([stats])
    stats_summary.to_excel(writer, sheet_name='Statistics', index=False)
    
    # Cluster size distribution
    cluster_dist_df.to_excel(writer, sheet_name='Cluster_Distribution', index=False)
    
    # Municipality statistics
    if 'municipalityName' in df.columns:
        muni_df.to_excel(writer, sheet_name='Municipality_Stats', index=False)

print("Results saved successfully!")
print(f"\nOutput file contains:")
print(f"  - Sheet 'Cleaned': {len(cleaned_df)} stations")
print(f"  - Sheet 'Removed': {len(removed_df)} stations")


 Saving results to: /Users/geo/Desktop/fuelstation-detection-thesis/data/ALL_cleaned.xlsx
Results saved successfully!

Output file contains:
  - Sheet 'Cleaned': 1026 stations
  - Sheet 'Removed': 236 stations
